In [5]:
import os
import sys
import csv
import glob
import datetime
import shutil
import random
import subprocess
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

In [123]:
# ~2min
human_gff = "/data/database/GRCh38/GENCODE/gencode.v27.annotation.gff3"
# human_gff = "/home/galaxy/data/splicing_sites/test.gff3"
names = ["Chr", "DB", "Feature", "Start", "End", "Dot_1", "Strand", "Dot_2", "Info"]
df = pd.read_table(human_gff, sep="\t", header=None, names=names, comment="#")
# print(df.head())
df["Gene_id"] = df["Info"].str.split("gene_id=").str[1].str.split(";").str[0]
df["Transcript_id"] = df["Info"].str.split("transcript_id=").str[1].str.split(";").str[0]
df_exon = df[df["Feature"] == "exon"]
# print(df_exon.head())
df_exon["Transcript_id"] = df_exon["Gene_id"] + ";" + df_exon["Transcript_id"]
df_bed = df_exon[["Chr", "Start", "End", "Strand", "Transcript_id"]]
print(df_bed.head())

    Chr  Start    End Strand                        Transcript_id
2  chr1  11869  12227      +  ENSG00000223972.5;ENST00000456328.2
3  chr1  12613  12721      +  ENSG00000223972.5;ENST00000456328.2
4  chr1  13221  14409      +  ENSG00000223972.5;ENST00000456328.2
6  chr1  12010  12057      +  ENSG00000223972.5;ENST00000450305.2
7  chr1  12179  12227      +  ENSG00000223972.5;ENST00000450305.2


/home/galaxy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [124]:
# run time:~20min
df_result = df_bed.groupby(["Transcript_id"]).apply(get_splice_site)
print(df_result.head())

                                         Chr      Start        End Strand  \
Transcript_id                                                               
ENSG00000000003.14;ENST00000373020.8 0  chrX  100636605  100636607      -   
                                     1  chrX  100635746  100635748      -   
                                     2  chrX  100635555  100635557      -   
                                     3  chrX  100635252  100635254      -   
                                     4  chrX  100635175  100635177      -   

                                                               Transcript_id  \
Transcript_id                                                                  
ENSG00000000003.14;ENST00000373020.8 0  ENSG00000000003.14;ENST00000373020.8   
                                     1  ENSG00000000003.14;ENST00000373020.8   
                                     2  ENSG00000000003.14;ENST00000373020.8   
                                     3  ENSG00000000003.14;E

In [117]:
# attention! gff3 is one-based, while bed is zero-based format.
def get_splice_site(df_i):
    splice_list = []
    df_i = df_i.drop_duplicates()
    if len(df_i) > 1:
        # first exon
        exon_1 = df_i.iloc[0, :]
        res = []
        if exon_1["Strand"] == "+":
            res = [exon_1["Chr"],exon_1["End"],exon_1["End"]+2,exon_1["Strand"],exon_1["Transcript_id"],"5"]
        else:
            res = [exon_1["Chr"],exon_1["Start"]-3,exon_1["Start"]-1,exon_1["Strand"],exon_1["Transcript_id"],"3"]
        splice_list.append(res)
        # middle exon
        if len(df_i) > 2:
            for i in range(1, len(df_i)-1):
                exon_m = df_i.iloc[i, :]
                if exon_1["Strand"] == "+":
                    splice_3=[exon_m["Chr"],exon_m["Start"]-3,exon_m["Start"]-1,exon_m["Strand"],exon_m["Transcript_id"],"3"]
                    splice_5=[exon_m["Chr"],exon_m["End"],exon_m["End"]+2,exon_m["Strand"],exon_m["Transcript_id"],"5"]
                else:
                    splice_3=[exon_m["Chr"],exon_m["End"],exon_m["End"]+2,exon_m["Strand"],exon_m["Transcript_id"],"5"]
                    splice_5=[exon_m["Chr"],exon_m["Start"]-3,exon_m["Start"]-1,exon_m["Strand"],exon_m["Transcript_id"],"3"]
                splice_list.append(splice_3)
                splice_list.append(splice_5)
        # last exon
        exon_f = df_i.iloc[-1, :]
        if exon_1["Strand"] == "+":
            res = [exon_f["Chr"],exon_f["Start"]-3,exon_f["Start"]-1,exon_f["Strand"],exon_f["Transcript_id"],"3"]
        else:
            res = [exon_f["Chr"],exon_f["End"],exon_f["End"]+2,exon_f["Strand"],exon_f["Transcript_id"],"5"]
        splice_list.append(res)
    df_splice = pd.DataFrame(splice_list, columns=["Chr", "Start", "End", "Strand", "Transcript_id", "Event"])
    df_splice["Start"] = df_splice["Start"].astype(int)
    df_splice["End"] = df_splice["End"].astype(int)
    return df_splice

In [125]:
df_result["name"]=df_result["Transcript_id"]+";"+df_result["Chr"]+":"+df_result["Start"].astype(str)+"-"+df_result["End"].astype(str)+";"+df_result["Event"]
df_result["dot"] = "."
df_final = df_result[["Chr", "Start", "End", "name", "dot", "Strand"]]
df_final.to_csv("/home/galaxy/data/splicing_sites/splice_sites.bed", sep="\t", header=False, index=False)

In [131]:
get_fasta("/home/galaxy/data/splicing_sites/splice_sites.bed")

In [130]:
def get_fasta(bed):
    result_file = "/home/galaxy/data/splicing_sites/splice_sites.fa"
    reference_genome = "/data/database/GRCh38/GENCODE/GRCh38.primary_assembly.genome.fa"
    command = "bedtools getfasta -fi %s -bed %s -s -name" % (reference_genome, bed)
    sub_p = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    fa_sequence = str(sub_p.communicate()[0].decode("utf-8"))
    with open(result_file, 'w') as fw:
        fw.write(fa_sequence)
# return fa_sequence

In [132]:
# combine bed and sequence
data_dir = "/home/galaxy/data/splicing_sites"
bed = os.path.join(data_dir, "splice_sites.bed")
fa = os.path.join(data_dir, "splice_sites.fa")
result_file = os.path.join(data_dir, "splice_sites.txt")
####################################################
df_bed = pd.read_table(bed, sep="\t", header=None, names=["chr", "start", "end", "name", "dot", "strand"])
# print(df_bed.head())
# df_fa = pd.read_table(fa, sep="\t", header=None)
name_list, nucleo_list = [], []
with open(fa, 'r') as f:
    contents = f.readlines()
    for i in range(0, len(contents), 2):
        name_list.append(contents[i].split(">")[1].strip().split("(")[0])
        nucleo_list.append(contents[i+1].strip())
df_fa = pd.DataFrame({"name": name_list, "Nucle": nucleo_list})
# print(df_fa.head())
df = df_bed.merge(df_fa, on="name", how="left")
print(df.head())
df.to_csv(result_file, sep="\t", index=False)

    chr      start        end  \
0  chrX  100636605  100636607   
1  chrX  100635746  100635748   
2  chrX  100635555  100635557   
3  chrX  100635252  100635254   
4  chrX  100635175  100635177   

                                                name dot strand Nucle  
0  ENSG00000000003.14;ENST00000373020.8;chrX:1006...   .      -    GT  
1  ENSG00000000003.14;ENST00000373020.8;chrX:1006...   .      -    AG  
2  ENSG00000000003.14;ENST00000373020.8;chrX:1006...   .      -    GT  
3  ENSG00000000003.14;ENST00000373020.8;chrX:1006...   .      -    AG  
4  ENSG00000000003.14;ENST00000373020.8;chrX:1006...   .      -    GT  


In [134]:
## filter splice sites based on nucleotide
## This is method needs to be check!!!!!!!!!!!!!!!!!!
data_dir = "/home/galaxy/data/splicing_sites/"
nucleo_file = os.path.join(data_dir, "splice_sites.txt")
res_file = os.path.join(data_dir, "filtered_splice_sites.txt")
###########################################################
result_list = []
with open(nucleo_file, 'r') as f:
    title = f.readline()
    contents = f.readlines()
    for i in range(0, len(contents), 2):
        if contents[i].strip().split("\t")[-1] == "GT" and contents[i+1].strip().split("\t")[-1] == "AG":
            result_list.append(contents[i])
            result_list.append(contents[i+1])
with open(res_file, 'w') as fw:
    result_list= ["%s\n" % x.strip() for x in result_list]
    fw.writelines(result_list)

In [2]:

##############
# 100bp upstream of 5SS, 100bp afterstream of 3SS
#
#                  up                down  
# Strand: -, [end, end+100], [start-99, start+1]
# Strand: +, [start-99, start+1], [end, end+100]
# Consider intron
# Strand: -, [end-100, end+100], [start-99, start+101]
# Strand: +, [start-99, start+101], [end-100, end+100]
##############
sp_file = "/home/galaxy/data/splicing_sites/filtered_splice_sites.txt"
result_file = "/home/galaxy/data/splicing_sites/splice_sites_flank_withIntron.bed"
##############
df_sp = pd.read_table(sp_file, sep="\t", names=["chr", "start", "end", "info", "dot", "strand", "nucleotide"])
df_sp["Type"] = df_sp["info"].str.split(";").str[-1]
df_sp["info"] = df_sp["info"] + ';' + df_sp["nucleotide"]
# print(df_sp.head())
#
df_a = df_sp[(df_sp["Type"]=="5") & (df_sp["strand"]=="+")]
df_a["new_start"], df_a["new_end"] = df_a["start"]-99, df_a["start"]+101
df_b = df_sp[(df_sp["Type"]=="5") & (df_sp["strand"]=="-")]
df_b["new_start"], df_b["new_end"] = df_b["end"]-100, df_b["end"]+100
df_c = df_sp[(df_sp["Type"]=="3") & (df_sp["strand"]=="+")]
df_c["new_start"], df_c["new_end"] = df_c["end"]-100, df_c["end"]+100
df_d = df_sp[(df_sp["Type"]=="3") & (df_sp["strand"]=="-")]
df_d["new_start"], df_d["new_end"] = df_d["start"]-99, df_d["start"]+101
df_list = []
for df in [df_a, df_b, df_c, df_d]:
    df_list.append(df[["chr", "new_start", "new_end", "info", "dot", "strand"]])
df_bed = pd.concat(df_list)
df_bed.to_csv(result_file, sep="\t", header=False, index=False)

#### Density Analysis

In [6]:
####################################################################################
eqtl_dir = "/home/galaxy/data/QTL/m6a_related_SNPs/eQTL/m6a_qtl_bed/"
eqtl_list = glob.glob("%s/*.bed" % eqtl_dir)
flank_file = "/home/galaxy/data/splicing_sites/splice_sites_flank_withIntron.bed"
result_dir = "/home/galaxy/data/splicing_sites/density_analysis"
#################################################################################
def distinguish_orientation():
    df = pd.read_table(flank_file, sep="\t", header=None, names=["chr", "start", "end", "info", "dot", "strand"])
    df["Type"] = df["info"].str.split(";").str[-2]  # !
    df_5, df_3 = df[df["Type"]=="5"], df[df["Type"]=="3"]
    return df_5, df_3

def split_into_bins(df):
    n_arrays = []
    for x,values in df.iterrows():
        bin_list,chr_n,start,end,strand =[],values["chr"],int(values["start"]),int(values["end"]),values["strand"]
        for i in range(start, end, 20):
            bin_list.append([chr_n, i, i+20])
        # reverse - strand order
        if strand == "+":
            n_arrays.append(bin_list)
        else:
            bin_list.reverse()
            n_arrays.append(bin_list)
    return n_arrays

def transform_into_bed(n_arrays):
    total_bins = []
    for j in range(len(n_arrays[0])):
        bin_str, bin_list = "", []
        for i in range(len(n_arrays)):
            i_list = n_arrays[i][j]
            str_bed = "\t".join([str(x) for x in i_list])
            bin_list.append(str_bed)
        # each bin
        bin_str = "\n".join(bin_list)
        total_bins.append(bin_str)
    return total_bins
            

def overlap_with_snp(eqtl, str_bin):
    command = "bedtools intersect -a stdin -b %s -wa | wc -l" % eqtl
    sub_p = subprocess.Popen(command, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    str_bin = bytes(str_bin, 'utf-8')
    snp_num = int(sub_p.communicate(str_bin)[0].decode("utf-8").strip())
    return snp_num
    
def main_method():
    df_5, df_3 = distinguish_orientation()
    array_5, array_3 = split_into_bins(df_5), split_into_bins(df_3)
    str5_list, str3_list = transform_into_bed(array_5), transform_into_bed(array_3)
    print("Pre process has done!")
    for eqtl in eqtl_list:
        print(os.path.basename(eqtl))
        str5_num = [overlap_with_snp(eqtl, str5) for str5 in str5_list]
        print(str5_num)
        str3_num = [overlap_with_snp(eqtl, str3) for str3 in str3_list]
        print(str3_num)
#         result_file_up = os.path.join(result_dir, "up-%s" % os.path.basename(eqtl))
#         result_file_down = os.path.join(result_dir, "down-%s" % os.path.basename(eqtl))
#         df_result_up.to_csv(result_file_up, sep="\t", header=False, index=False)
#         df_result_down.to_csv(result_file_down, sep="\t", header=False, index=False)

main_method() # eqtl_list[0]

Pre process has done!
Brain_Nucleus_accumbens_basal_ganglia.v7.signif_variant_gene_pairs.bed-Brain.bed
[325, 421, 490, 692, 301, 300, 286, 556, 537, 885]
[250, 421, 365, 382, 345, 471, 442, 476, 376, 231]
Heart_Left_Ventricle.v7.signif_variant_gene_pairs.bed-Heart.bed
[386, 589, 486, 593, 459, 642, 461, 543, 568, 451]
[416, 508, 529, 446, 441, 422, 686, 669, 580, 369]
Brain_Substantia_nigra.v7.signif_variant_gene_pairs.bed-Brain.bed
[55, 114, 154, 339, 97, 101, 108, 89, 171, 326]
[58, 192, 92, 99, 119, 161, 116, 243, 92, 79]
Brain_Hippocampus.v7.signif_variant_gene_pairs.bed-Brain.bed
[175, 210, 251, 427, 145, 145, 208, 252, 353, 567]
[151, 337, 198, 189, 158, 285, 268, 317, 195, 122]
Heart_Atrial_Appendage.v7.signif_variant_gene_pairs.bed-Heart.bed
[389, 534, 599, 754, 514, 610, 486, 656, 663, 588]
[461, 556, 543, 447, 466, 592, 781, 693, 704, 406]
Brain_Hypothalamus.v7.signif_variant_gene_pairs.bed-Brain.bed
[160, 188, 272, 515, 184, 138, 219, 227, 347, 578]
[123, 208, 212, 223, 145,

In [156]:
result_dir = "/home/galaxy/data/splicing_sites/density_analysis"
result_file_up = os.path.join(result_dir, "down-Brain_Putamen_basal_ganglia.v7.signif_variant_gene_pairs.bed-Brain.bed")
result_file_down = os.path.join(result_dir, "splice_sites_flank_down.bed")
df_up = pd.read_table(result_file_up, sep="\t")
df_down = pd.read_table(result_file_down, sep="\t")
# for x in df_up:
#     print(sum(df_up[x]))
print(df_up.sum(0))
# print("###################")
# print(df_down.sum(0))

0.0      151.0
0.0.1    188.0
0.0.2    270.0
0.0.3    262.0
0.0.4    257.0
dtype: float64
